In [ ]:
#imports
import numpy as np
import pandas as pd
import os
import glob


Print switch:
switch to true to create csv files


In [ ]:
switch  =  False

Get files and filepaths


In [ ]:
directory = '/Users/chriskimmons/My_Projects/NYT_COV/covidCSV/'

covid_case_files = list(glob.glob(os.path.join(directory,'*.*')))
covid_case_files.sort()

print(covid_case_files)

Load all 4 csv files and append to DF


In [ ]:
all_counties_df = pd.DataFrame(columns=['date','state','county','fips','cases','deaths'])

for file in covid_case_files[8:12]:
    all_counties_df = pd.concat([all_counties_df, pd.read_csv(file, parse_dates=['date'])], ignore_index=True)

all_counties_df.cases = pd.to_numeric(all_counties_df.cases)

all_counties_df.info()
# get unique values from county and state columns
null_counties = all_counties_df.county.loc[all_counties_df.fips.isnull()].unique()

null_states = all_counties_df.state.loc[all_counties_df.fips.isnull()].unique()

print(null_counties,'',null_states, sep='\n')
# view nulls in all_counties

# convert deaths nulls to 0 and convert to int
all_counties_df = all_counties_df.fillna(0).astype({'fips':'int','deaths':'int'})
all_counties_df.isnull().sum()


In [ ]:
# Find counties of islands that have county level data
list_of_island_counties = (all_counties_df.county
                           .loc[all_counties_df.state
                                .isin(['American Samoa',
                                       'Guam',
                                       'Northern Mariana Islands',
                                       'Virgin Islands'])]
                                .unique())
list_of_island_counties

Import island territory tables

In [ ]:

# import fips table
fips_df = (pd.read_csv('/Users/chriskimmons/My_Projects/NYT_COV/covidCSV/US_fips_codes.csv',
                       index_col=[0])
                .rename(columns={"FIPS":'fips','County or equivalent': 'county', 'State or equivalent': 'state'})
                .astype({'fips':'str'}))

fips_df.county = fips_df.county.str.replace(r'\[.*?\]', '', regex=True)

# import mainland census table
census_mainland = covid_case_files[3]

df_mainland = (pd.read_csv(census_mainland, 
                          encoding='latin-1',
                          usecols=['STATE','COUNTY','STNAME','CTYNAME','ESTIMATESBASE2020'],
                          dtype={'STATE':'str','COUNTY':'str',})
                          .rename(columns={'STATE':'state_id',
                                           'COUNTY':'county_id',
                                           'STNAME':'state',
                                           'CTYNAME':'county',
                                           'ESTIMATESBASE2020':'base2020'}))

df_mainland['fips'] = (df_mainland.state_id + df_mainland.county_id).astype('int')
df_mainland = df_mainland.drop(columns=['state_id','county_id'])
df_mainland.head()
#########


In [ ]:

# Index finding tool for .at() and .loc()
def index_finder(df,column,value):
    return df.loc[df[f'{column}'] == value].index[0]


In [ ]:

# import Samoa table
df_samoa = (pd.read_csv(covid_case_files[2],
                        encoding='latin-1', 
                        skiprows=3, 
                        header=0, 
                        usecols=['Geographic area','2020'])
            .dropna(axis=1,how='all')
            .dropna(axis=0,how='any')
            .rename(columns={'Geographic area':'county', '2020':'base2020'}))

df_samoa['state'] = 'American Samoa'
df_samoa = df_samoa.reindex(columns=['state', 'county', 'base2020'])

df_samoa.at[16,'county'] = "Swain's Island"
df_samoa.at[15,'county'] = 'Rose Atoll (Rose Island)'

df_samoa.at[index_finder(df_samoa,'county',"American Samoa"), 'base2020']

# merge Samoa and Fips
df_samoa = pd.merge(left=df_samoa, right=fips_df[['county', 'fips']], on='county', how='left')
print(len(df_samoa))


df_samoa.at[index_finder(df_samoa,'county',"American Samoa"), 'fips'] = 60000
df_samoa.head()

In [ ]:
# Import Guam table
df_guam = (pd.read_csv(covid_case_files[5],
                       encoding='latin-1', 
                       skiprows=3, 
                       header=0, 
                       usecols=['Geographic area', '2020'])
           .dropna(axis=0,how='all')
           .dropna(axis=0,how='any')
           .rename(columns={'Geographic area':'county','2020':'base2020'}))

df_guam = df_guam.loc[df_guam['county'] != 'County subdivision not defined']
df_guam['state'] = 'Guam'

# merge Guam and Fips
df_guam = pd.merge(left=df_guam, right=fips_df[['county', 'fips']], on='county', how='left')
print(len(df_guam))

df_guam

In [ ]:
# Import PR (already include fips data)
df_pr = (pd.read_csv(covid_case_files[0],
                    encoding='latin-1',
                    usecols=['STATE','COUNTY','STNAME','CNTYNAME','POPESTIMATE2020'],
                    dtype={'STATE':'str','COUNTY':'str',})
                    .rename(columns={'STATE':'state_id','COUNTY':'county_id','STNAME':'state','CNTYNAME':'county','POPESTIMATE2020':'base2020'}))

df_pr['fips'] = (df_pr.state_id + df_pr.county_id).astype('int')
df_pr = df_pr.drop(columns=['state_id','county_id'])

df_pr.head()

In [ ]:
# Import VI
df_vi = (pd.read_csv(covid_case_files[13],
                     encoding='latin-1',
                     skiprows=3,
                     header=0,
                     usecols=['Geographic area','2020'])
         .dropna(axis=1, how='all')
         .dropna(axis=0, how='any')
         .rename(columns={'Geographic area':'county','2020':'base2020'}))

df_vi = df_vi.loc[df_vi['county'] != 'Island subdivision not defined']
df_vi['state'] = 'Virgin Islands'
df_vi.county = df_vi.county.str.replace(r'St\.', 'Saint', regex=True)
df_vi
# merge VI and Fips
df_vi = pd.merge(left=df_vi, right=fips_df[['county', 'fips']], on='county', how='left')
print(len(df_vi))

df_vi.county = df_vi.county.str.replace(r'\bSaint\b', 'St.', regex=True)
df_vi.at[index_finder(df_vi,'county', 'United States Virgin Islands'), 'fips'] = 69000
df_vi.at[index_finder(df_vi,'county', 'United States Virgin Islands'), 'county'] = 'Virgin Islands'

df_vi

In [ ]:
# Import Marianas table
df_marianas = (pd.read_csv(covid_case_files[4],
                           encoding='latin-1',
                           skiprows=4,
                           header=0,
                           usecols=['Geographic area', '2020'])
               .dropna(axis=1,how='all')
               .dropna(axis=0,how='any')
               .rename(columns={'Geographic area':'county', '2020':'base2020'}))

df_marianas = df_marianas.loc[df_marianas['county'] != 'Municipality subdivision not defined']

df_marianas['state'] = 'Northern Mariana Islands'
# merge marianas and fips
df_marianas = pd.merge(left=df_marianas, right=fips_df[['county', 'fips']], on='county', how='left')
print(len(df_marianas))

df_marianas.at[index_finder(df_marianas,'county', 'Commonwealth of the Northern Mariana Islands'), 'county'] = 'Northern Mariana Islands'
df_marianas.at[index_finder(df_marianas,'county', 'Northern Mariana Islands'), 'fips'] = 69000

df_marianas.head()

In [ ]:
# build full fips table with concat
full_fips_df = pd.concat([df_mainland,df_pr,df_samoa,df_guam,df_vi,df_marianas], ignore_index=True)


full_fips_df.base2020 = full_fips_df.base2020.replace(',','', regex=True)
full_fips_df = (full_fips_df.dropna(axis=0, how='any')
                .astype({'base2020':'int','fips':'int'}))

full_fips_df.head()

Data Cleaning

In [ ]:
# function to aggregate county populations to match NYT data
def aggregate_populations(list_of_states,list_of_counties):
    return (full_fips_df.loc[(full_fips_df.state.isin(list_of_states)) & 
                             (full_fips_df.county.isin(list_of_counties))]
                             .base2020.sum())

# find population of given state
def get_state_population(state):
    population_row =full_fips_df.loc[full_fips_df.state == state, 'base2020']
    return population_row[population_row.index[0]]


In [ ]:

# Fixing specific data points

specific_case_tuples = [
                        ('Missouri', 'Joplin Area', 0, 37592),
                        ('Missouri', 'Kansas City Area', 0, 3800)
                        ]

# populations are set to 0 to avoid doubling with tableau's calculations

# aggregate regions
county_aggs = {'New_York_City_counties':['New York County', 'Kings County', 'Queens County', 'Bronx County', 'Richmond County'],
               'Valdez_Cordova_counties':['Copper River Census Area','Chugach Census Area'],
               'Bristol_Bay_and_Lake_counties':['Bristol Bay Borough','Lake and Peninsula Borough'],
               'Yakutat':['Yakutat City and Borough','Hoonah-Angoon Census Area']
               }

specific_case_tuples.extend([('New York','New York City',0, 51000),
                      ('Alaska','Valdez-Cordova Census Area', aggregate_populations(['Alaska'],county_aggs['Valdez_Cordova_counties']), 2261),
                      ('Alaska', 'Bristol Bay plus Lake and Peninsula', 0, 2997),
                      ('Alaska', 'Yakutat plus Hoonah-Angoon', 0,2998)
                      ])

specific_case_tuples = pd.DataFrame(specific_case_tuples,columns=['state','county','base2020','fips'])
print(specific_case_tuples.info())

full_fips_df = pd.concat([full_fips_df,specific_case_tuples], ignore_index=True)

full_fips_df.tail(20)

In [ ]:
# separate states from counties for states table
state_pop_df = full_fips_df.query('county == state').copy()

state_pop_df.fips = state_pop_df.fips * 0.001
state_pop_df = (state_pop_df
                .astype({'fips':'int'})
                .drop(columns=['county'])
                .drop_duplicates()
                .reset_index(drop=True))
                
state_pop_df['state_id'] = state_pop_df.index + 1
state_pop_df = state_pop_df.reindex(columns=['state_id','state','fips','base2020'])

# export State CSV
if switch:
    state_pop_df.to_csv('/Users/chriskimmons/My_Projects/NYT_COV/star_schema_files/dimState.csv', index=False)
    print('file created')

In [ ]:
county_pop_df = full_fips_df.query('county != state').copy()
county_pop_df = county_pop_df.reset_index(drop=True)
print(len(county_pop_df))

county_pop_df.county = (county_pop_df.county.str.replace('á','a')
                       .str.replace('é','e')
                       .str.replace('í','i')
                       .str.replace('ó','o')
                       .str.replace('ü','u')
                       .str.replace('ñ','n'))


# # Fixing specific data points

specific_case_tuples = [('District of Columbia', 'District of Columbia', get_state_population('District of Columbia'), 11001)]

specific_case_tuples

In [ ]:

# # add special case tuples to county data
specific_case_tuples = pd.DataFrame(specific_case_tuples,columns=['state','county','base2020','fips'])
print(specific_case_tuples.info())

county_pop_df = pd.concat([county_pop_df,specific_case_tuples], ignore_index=True)

# Merge county df and state df to creat state_id (check here for bugs)
county_pop_df = pd.merge(county_pop_df,state_pop_df[['state_id','state']],on='state', validate='many_to_one')
county_pop_df = county_pop_df.reindex(columns=['county','state_id','state','fips','base2020'])
county_pop_df
unknown_df = pd.DataFrame(null_states, columns=['state'])
unknown_df['county'] = 'Unknown'

unknown_df = pd.merge(left=unknown_df,right=state_pop_df,on='state')
unknown_df = unknown_df.sort_values('state_id')
county_pop_df = pd.concat([county_pop_df,unknown_df],ignore_index=True)

# Rename Guam and Samoa for query purposes in dataset
county_pop_df.county = np.where(county_pop_df.state.isin(['Guam','American Samoa']), county_pop_df.state, county_pop_df.county)

# build county ID column
county_pop_df['county_id'] = county_pop_df.index + 1

# sort ids to build county_id
county_pop_df = county_pop_df.sort_values('county_id').reset_index(drop=True)
county_pop_df['county_id'] = county_pop_df.index

# finally, we clean the 'Pending County Assignment'
all_counties_df = all_counties_df.drop(all_counties_df[all_counties_df.county=='Pending County Assignment'].index)

# fill unknown county with state name (HERE!!!!)
all_counties_df.county = np.where(all_counties_df.state.isin(['Guam','American Samoa']), all_counties_df.state, all_counties_df.county)

# remove extra language from counties, reset where appropriate
replace_county_labels = '|'.join([' County',' Municipality',' Island',' Municipio',' Parish'])
county_pop_df.county = county_pop_df.county.str.replace(replace_county_labels,'',regex=True, case=False)

county_pop_df.county.loc[county_pop_df.county == 'Dona Ana'] = 'Doña Ana'

county_pop_df.county.loc[county_pop_df.county == 'Skagway'] = 'Skagway Municipality'

county_pop_df.county.loc[county_pop_df.county == 'Kodiak Borough'] = 'Kodiak Island Borough'

county_pop_df.county.loc[(county_pop_df.county == 'Rock') & (county_pop_df.state == 'Illinois')] = 'Rock Island'

county_pop_df = county_pop_df.reindex(columns=['county_id','county','state_id','state','fips','base2020'])
# create county CSV
if switch:
    county_pop_df.to_csv('/Users/chriskimmons/My_Projects/NYT_COV/star_schema_files/dimCounty.csv',index=False)
    print('file created')


Create Date table

In [ ]:
# find unque dates
unique_dates = pd.Series(all_counties_df.date.unique())
unique_dates   
# create date df
date_table = pd.DataFrame(columns=['date_id', 'full_date','year','quarter','quarter_name','month','month_name','day','day_of_week','day_of_year'])

# fill columns

date_table.date_id = unique_dates.index + 1
date_table.full_date = unique_dates
date_table.year = unique_dates.dt.year
date_table.quarter = unique_dates.dt.quarter
date_table.quarter_name = 'Q'+ unique_dates.dt.quarter.astype(str)
date_table.month = unique_dates.dt.month
date_table.month_name = unique_dates.dt.month_name()
date_table.day = unique_dates.dt.day
date_table.day_of_week = unique_dates.dt.day_of_week
date_table.day_of_year = unique_dates.dt.day_of_year

# create date CSV
if switch:
    date_table.to_csv('/Users/chriskimmons/My_Projects/NYT_COV/star_schema_files/dimDate.csv', index=False)
    print('file created')


Fact table

In [ ]:
def city_aggregator(df,
                    state,
                    list_of_counties,
                    aggregated_region_name, 
                    fips=0):
    
    filtered_counties = df.loc[(df.state == f'{state}') & 
                               (df.county.isin(list_of_counties))]
    counties_aggregated = (filtered_counties
                           .groupby('date')
                           .sum())
    counties_aggregated['fips'] = fips
    counties_aggregated['county'] = aggregated_region_name
    counties_aggregated['state'] = state

    counties_aggregated = counties_aggregated.reset_index()
    df  = (pd.concat([df,
                      counties_aggregated],
                     ignore_index=True)
           .sort_values('date')
           .reset_index(drop=True))
    filtered_counties = df.loc[(df.state == f'{state}') & 
                               (df.county.isin(list_of_counties))]
    
    df = df.drop(filtered_counties.index)
    # print(df.loc[filtered_counties.index])
    return df


In [ ]:
all_counties_df = city_aggregator(all_counties_df,
                                  'Missouri',
                                  ['Jasper','Newton','Joplin'],
                                   'Joplin Area',
                                   fips=37592)

print(all_counties_df.query('county == "Joplin Area"').head(1),
      " ",
      all_counties_df.query('county == "Jasper" & state == "Missouri"').head(1),
      sep='\n')
all_counties_df = city_aggregator(all_counties_df,
                                  'Missouri',
                                  ['Cass','Clay','Platte','Jackson','Kansas City'],
                                   'Kansas City Area',
                                   fips=38000)

all_counties_df = all_counties_df.drop(columns=['fips'])

all_counties_df = all_counties_df.merge(county_pop_df[['state','county','fips']],on=['state','county'],how='left')

# copy table from all counties
fact_table = all_counties_df.copy()

# merge with fips table (look here for bugs)

fact_table = (fact_table
              .merge(county_pop_df[['fips','state_id','county_id']],left_on='fips',right_on='fips', how='left')
              .merge(date_table[['full_date','date_id']],left_on='date',right_on='full_date')
              )

fact_table['case_id'] = fact_table.index + 1


fact_table = fact_table.drop(columns=['date','full_date','state','county','fips'])

# reindex columns for export
fact_table = fact_table.reindex(columns=['case_id','date_id','state_id','county_id','cases','deaths'])

# create fact CSV
if switch:
    fact_table.to_csv('/Users/chriskimmons/My_Projects/NYT_COV/star_schema_files/factCases.csv', index=False)
    print('file created')
